<a href="https://colab.research.google.com/github/MidnightTourist/project-point-nemo/blob/main/Copy_of_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

class PopularityBasedModel:
    """
    Popularity-based recommendation system for cold start scenarios.
    Ranks items by recent weighted interactions.
    """

    def __init__(self, time_window_days=30, weights=None):
        """
        Args:
            time_window_days: Number of days to consider for recency (default: 30)
            weights: Dict with interaction type weights, e.g.,
                    {'click': 1.0, 'add_to_cart': 3.0, 'watch_time': 0.01}
        """
        self.time_window_days = time_window_days
        self.weights = weights or {
            'click': 1.0,
            'add_to_cart': 3.0,
            'watch_time': 0.01  # per second
        }
        self.popularity_scores = None
        self.item_rankings = None

    def fit(self, interactions_df):
        """
        Calculate popularity scores from interaction data.

        Args:
            interactions_df: DataFrame with columns:
                - item_id: video/product identifier
                - interaction_type: 'click', 'add_to_cart', or 'watch_time'
                - value: interaction value (1 for click/cart, seconds for watch_time)
                - timestamp: datetime of interaction
        """
        # Filter to recent interactions
        cutoff_date = datetime.now() - timedelta(days=self.time_window_days)
        recent_interactions = interactions_df[
            interactions_df['timestamp'] >= cutoff_date
        ].copy()

        # Calculate weighted scores using TensorFlow for scalability
        item_scores = {}

        for item_id in recent_interactions['item_id'].unique():
            item_data = recent_interactions[
                recent_interactions['item_id'] == item_id
            ]

            # Calculate weighted score
            score = 0.0
            for interaction_type, weight in self.weights.items():
                type_data = item_data[
                    item_data['interaction_type'] == interaction_type
                ]
                if len(type_data) > 0:
                    score += weight * type_data['value'].sum()

            item_scores[item_id] = score

        # Convert to TensorFlow tensors for efficient computation
        self.item_ids = tf.constant(list(item_scores.keys()))
        self.popularity_scores = tf.constant(list(item_scores.values()),
                                            dtype=tf.float32)

        # Create rankings
        sorted_indices = tf.argsort(self.popularity_scores, direction='DESCENDING')
        self.item_rankings = tf.gather(self.item_ids, sorted_indices)

        return self

    def recommend(self, n_items=10, exclude_items=None):
        """
        Get top N popular items.

        Args:
            n_items: Number of recommendations to return
            exclude_items: List of item IDs to exclude (e.g., already purchased)

        Returns:
            List of recommended item IDs
        """
        if self.item_rankings is None:
            raise ValueError("Model not fitted. Call fit() first.")

        rankings = self.item_rankings.numpy()

        if exclude_items:
            exclude_set = set(exclude_items)
            rankings = [item for item in rankings if item not in exclude_set]

        return rankings[:n_items]

    def get_score(self, item_id):
        """Get popularity score for a specific item."""
        idx = tf.where(self.item_ids == item_id)
        if tf.size(idx) == 0:
            return 0.0
        return self.popularity_scores[idx[0][0]].numpy()


# Example usage:
if __name__ == "__main__":
    # Sample data
    data = {
        'item_id': ['video_1', 'video_1', 'video_2', 'video_3', 'video_1',
                    'video_2', 'video_3', 'video_4'],
        'interaction_type': ['click', 'watch_time', 'click', 'add_to_cart',
                            'add_to_cart', 'watch_time', 'click', 'click'],
        'value': [1, 120, 1, 1, 1, 45, 1, 1],  # watch_time in seconds
        'timestamp': [datetime.now() - timedelta(days=i)
                     for i in [1, 1, 2, 3, 5, 7, 10, 15]]
    }

    interactions_df = pd.DataFrame(data)

    # Initialize and fit model
    model = PopularityBasedModel(
        time_window_days=30,
        weights={
            'click': 1.0,
            'add_to_cart': 3.0,
            'watch_time': 0.01
        }
    )

    model.fit(interactions_df)

    # Get recommendations
    recommendations = model.recommend(n_items=5)
    print("Top 5 Recommendations:", recommendations)

    # Get score for specific item
    score = model.get_score('video_1')
    print(f"Popularity score for video_1: {score}")